In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import resNeXt_pytorch
from torch.autograd.variable import Variable
import os
from tqdm import tqdm
import resNeXt_pytorch
import net

In [ ]:
learning_rate=None
toOneHot = True
dimension = 2

In [ ]:
def learning_rate_schedule(optimizer, epoch):
    global learning_rate
    lr = learning_rate * (0.1 ** int(epoch / 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def train(train_loader, model, lossfn, optimizer, epoch):
    model.train()
    for i, (input, target) in enumerate(train_loader):
        if toOneHot:
            target = toOneHot(dimension, target)
        target = target.cuda(non_blocking=True)
#         target = target.cpu()
        output = model(input)
        
        loss = lossfn(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Training Loss", loss)

In [ ]:
def validate(val_loader, model, lossfn):
    
    model.eval()
    accuracies = []
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if toOneHot:
                target = toOneHot(dimension, target)
            target = target.cuda(non_blocking=True)
#             target = target.cpu()

            output = model(input)
            loss = lossfn(output, target)

            acc1 = accuracy(output, target)
            accuracies.extend(acc1)
#             if i % 10 == 0:
#                 print( 'Top 1 accuracy: {:.2}, Top 5 accuracy: {:.2}'.format(acc1[0], acc5[0]))

    return np.mean(accuracies)

In [ ]:
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
def save_checkpoint(state, best, filename='checkpoint.tar'):
    torch.save(state, filename)
    if best:
        shutil.copyfile(filename, 'best_checkpoint.tar')

In [ ]:
def toOneHot(dimension, int_target):
    int_target = int_target.numpy()
    n = int_target.size
    one_hot_vector = np.zeros((n, dimension), dtype = 'float32')
    one_hot_vector[np.arange(n), int_target] = 1
    return torch.from_numpy(one_hot_vector)

In [ ]:
pretrained = False
distributed = False

# architecture = 'resnet18'
# print('creating model {}'.format(architecture))

model = resNeXt_pytorch.ResNeXt()
print(model)
# model = models.__dict__[architecture](pretrained=pretrained)

model = torch.nn.DataParallel(model).cuda()  #change this to .cuda() if gpu

In [ ]:
lossfn = nn.MSELoss().cuda()  #change this to .cuda() if gpu
global learning_rate
learning_rate = 0.008
momentum = 0.9
weight_decay = 1e-4
optimizer = torch.optim.Adam(model.parameters(),
                            learning_rate,
                            weight_decay=weight_decay)

In [ ]:
epochs = 40

In [ ]:
datadir = 'images'
traindir, valdir = os.path.join(datadir, 'train'), os.path.join(datadir, 'valid')
print('constructing training/validation datasets')


train_batch, valid_batch = 10, 4
train_workers, valid_workers = 1, 1
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.2, 0.2, 0.2])  # This should be computed on our images
transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                normalize,])

train_dataset, valid_dataset = datasets.ImageFolder(traindir, transform), datasets.ImageFolder(valdir, transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch, shuffle=True,
                                           num_workers=train_workers, pin_memory=False)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=valid_batch, shuffle = True,
                                         num_workers=valid_workers, pin_memory=False)


In [ ]:
best_precision = 0
print( 'training over {} epochs'.format(epochs) )
for epoch in range(epochs):
    print("Epoch", epoch)
    learning_rate_schedule(optimizer, epoch)

    train(train_loader, model, lossfn, optimizer, epoch)
    precision = validate(val_loader, model, lossfn)
    print("Validation Accuracy", precision)
    is_best = precision > best_precision
    best_precision = max(precision, best_precision)
    save_checkpoint({
        'epoch': epoch+1,
        'arch': architecture,
        'state_dict': model.state_dict(),
        'best_precision': best_precision,
        'optimizer' : optimizer.state_dict(),
    }, is_best)

<h2>Testing</h2>
<br>

In [ ]:
datadir = 'images'
testdir = os.path.join(datadir, 'test')

test_workers = 1

test_dataset = datasets.ImageFolder(testdir, transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset), shuffle = True,
                                          num_workers=test_workers, pin_memory=True)

In [ ]:
model.eval()
for data_batch, labels_batch in test_loader:

#     move to GPU if available
    
    data_batch, labels_batch = data_batch.cuda(async=True), labels_batch.cuda(async=True)
    print(type(labels_batch))
    target = labels_batch
#     fetch the next evaluation batch
#     data_batch, labels_batch = Variable(data_batch), Variable(labels_batch)

#     compute model output
    output_batch = model(data_batch)
#     loss = loss_fn(output_batch, labels_batch)
    _, pred = output_batch.topk(1, 1)
    pred = pred.t()

In [ ]:
target = target.cpu()
target = target.numpy()
prediciton = pred.reshape(-1).cpu()
prediction = prediciton.numpy()
if toOneHot:
    prediction = [np.where(r==1)[0][0] for r in prediction]
# assert (target.size() == prediction.size())

<h2>Confusion Matrix</h2>
<br>

In [ ]:

%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(target, prediction)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
cm_plot_labels = ['cat', 'dog']
plot_confusion_matrix(cm, cm_plot_labels, title = 'Confusion Matrix')